In [126]:
import pandas as pd

In [127]:
aggr = pd.read_csv('cluster_aggregates.csv')
aggr.head()

,Unnamed: 0,kmeans_119,moving_time,average_watts,suffer_score,ride_count
0,0,0,6776.846154,113.815385,82.153846,13
1,1,1,2048.645161,86.200000,25.806452,31
2,2,2,10094.000000,126.872727,65.414581,11
3,3,3,2204.875000,135.349603,33.853645,8
4,4,4,4053.000000,127.427273,68.037689,11


In [128]:
aggr.describe()

,Unnamed: 0,kmeans_119,moving_time,average_watts,suffer_score,ride_count
count,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
mean,59.000000,59.000000,5578.134597,137.794923,68.267114,12.075630
std,34.496377,34.496377,2559.163573,23.807952,21.072001,5.542073
min,0.000000,0.000000,1000.333333,80.923333,9.000000,1.000000
25%,29.500000,29.500000,3770.110909,125.687917,63.875000,9.000000
50%,59.000000,59.000000,5000.615385,139.109091,65.414581,11.000000
75%,88.500000,88.500000,7405.731618,155.655556,71.968750,14.000000
max,118.000000,118.000000,14002.555556,186.566667,141.500000,31.000000


In [129]:
pre_train = aggr[['moving_time', 'ride_count']]
pre_train.head()

,moving_time,ride_count
0,6776.846154,13
1,2048.645161,31
2,10094.000000,11
3,2204.875000,8
4,4053.000000,11


In [130]:
mean_avg_watts = aggr['average_watts'].mean()
mean_suffer_score = aggr['suffer_score'].mean()
pre_train.loc[:, 'average_watts'] = aggr['average_watts'].apply(lambda x : 1 if x >= mean_avg_watts else 0)
pre_train.loc[:, 'suffer_score'] = aggr['suffer_score'].apply(lambda x : 1 if x >= mean_suffer_score else 0)
pre_train.describe()

/var/folders/ph/n1bcmp_s2rlgrd9vg896nks80000gn/T/ipykernel_24727/9803172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train.loc[:, 'average_watts'] = aggr['average_watts'].apply(lambda x : 1 if x >= mean_avg_watts else 0)
/var/folders/ph/n1bcmp_s2rlgrd9vg896nks80000gn/T/ipykernel_24727/9803172.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train.loc[:, 'suffer_score'] = aggr['suffer_score'].apply(lambda x : 1 if x >= mean_suffer_score else 0)


,moving_time,ride_count,average_watts,suffer_score
count,119.000000,119.000000,119.000000,119.000000
mean,5578.134597,12.075630,0.521008,0.310924
std,2559.163573,5.542073,0.501671,0.464829
min,1000.333333,1.000000,0.000000,0.000000
25%,3770.110909,9.000000,0.000000,0.000000
50%,5000.615385,11.000000,1.000000,0.000000
75%,7405.731618,14.000000,1.000000,1.000000
max,14002.555556,31.000000,1.000000,1.000000


In [131]:
pre_train.head()
pre_train.to_csv('pre_classification_pre_plan.csv')

In [132]:
pre_train_plan = pre_train[['moving_time', 'ride_count']]
pre_train_plan.loc[:, 'training_plan'] = pre_train['average_watts'].astype(str).str.cat(pre_train['suffer_score'].astype(str), sep='_')
pre_train_plan.to_csv('pre_classification.csv')
pre_train_plan.head()

/var/folders/ph/n1bcmp_s2rlgrd9vg896nks80000gn/T/ipykernel_24727/4034469462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_train_plan.loc[:, 'training_plan'] = pre_train['average_watts'].astype(str).str.cat(pre_train['suffer_score'].astype(str), sep='_')


,moving_time,ride_count,training_plan
0,6776.846154,13,0_1
1,2048.645161,31,0_0
2,10094.000000,11,0_0
3,2204.875000,8,0_0
4,4053.000000,11,0_0


In [133]:
pre_train_plan.describe(include='all')

,moving_time,ride_count,training_plan
count,119.000000,119.000000,119
unique,NaN,NaN,4
top,NaN,NaN,0_0
freq,NaN,NaN,44
mean,5578.134597,12.075630,NaN
std,2559.163573,5.542073,NaN
min,1000.333333,1.000000,NaN
25%,3770.110909,9.000000,NaN
50%,5000.615385,11.000000,NaN
75%,7405.731618,14.000000,NaN


In [134]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

def train_test_classifiers(X_train, X_test, y_train, y_test):
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize classifiers
    classifiers = {
        'Logistic Regression': LogisticRegression(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Support Vector Machine': SVC(),
        'K-Nearest Neighbors': KNeighborsClassifier()
    }

    # Train and test each classifier
    for name, clf in classifiers.items():
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)

        # Evaluate the classifier
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='micro')

        # Display results
        print(f"\nClassifier: {name}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")

In [135]:
# Assuming 'training_plan' is your target variable
X = pre_train_plan[['moving_time', 'ride_count']]
y = pre_train_plan['training_plan']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [136]:
train_test_classifiers(X_train, X_test, y_train, y_test)


Classifier: Logistic Regression
Accuracy: 0.4722
F1 Score: 0.4722

Classifier: Decision Tree
Accuracy: 0.2778
F1 Score: 0.2778

Classifier: Random Forest
Accuracy: 0.4167
F1 Score: 0.4167

Classifier: Support Vector Machine
Accuracy: 0.4167
F1 Score: 0.4167

Classifier: K-Nearest Neighbors
Accuracy: 0.3611
F1 Score: 0.3611


In [137]:
pre_train_plan['training_plan'].value_counts()

training_plan
0_0    44
1_0    38
1_1    24
0_1    13
Name: count, dtype: int64

In [138]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit and apply SMOTE only to the training set
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled

(     moving_time  ride_count
 0    3402.111111           9
 1    9573.250000           4
 2    2773.909091          11
 3    1000.333333           3
 4    1873.153846          26
 ..           ...         ...
 119  5961.651778          13
 120  6621.485418          10
 121  4174.980962          13
 122  7993.387014          10
 123  6641.841022          10
 
 [124 rows x 2 columns],
 0      0_0
 1      1_0
 2      0_0
 3      0_0
 4      0_1
       ... 
 119    1_1
 120    1_1
 121    1_1
 122    1_1
 123    1_1
 Name: training_plan, Length: 124, dtype: object)

In [139]:
train_test_classifiers(X_train_resampled, X_test, y_train_resampled, y_test)


Classifier: Logistic Regression
Accuracy: 0.4444
F1 Score: 0.4444

Classifier: Decision Tree
Accuracy: 0.2778
F1 Score: 0.2778

Classifier: Random Forest
Accuracy: 0.2778
F1 Score: 0.2778

Classifier: Support Vector Machine
Accuracy: 0.3333
F1 Score: 0.3333

Classifier: K-Nearest Neighbors
Accuracy: 0.3611
F1 Score: 0.3611


In [140]:
import numpy as np

# Data Augmentation
augmented_data = pd.DataFrame()

# Define augmentation parameters
num_augmented_samples = 5  # Number of augmented samples per original sample

training_data = pd.concat([X_train, y_train], axis=1)

for index, row in training_data.iterrows():
    # Repeat the original row along with variations
    augmented_rows = pd.concat([pd.Series(row)] * num_augmented_samples, axis=1).T

    # Add noise or variation to the augmented rows
    # You can customize this based on your specific requirements
    augmented_rows['moving_time'] += np.random.normal(scale=0.1, size=num_augmented_samples)
    augmented_rows['ride_count'] += np.random.randint(-1, 2, size=num_augmented_samples)

    # Append the augmented rows to the new DataFrame
    augmented_data = pd.concat([augmented_data, augmented_rows])

# Display the augmented data
print("\nAugmented Data:")
augmented_data


Augmented Data:


,moving_time,ride_count,training_plan
62,3402.138306,8,0_0
62,3402.241408,9,0_0
62,3402.173201,8,0_0
62,3401.979884,9,0_0
62,3402.213546,10,0_0
...,...,...,...
102,5000.47815,13,1_1
102,5000.575756,14,1_1
102,5000.720474,12,1_1
102,5000.570067,13,1_1


In [141]:
X_train_aug = augmented_data[['moving_time', 'ride_count']]
y_train_aug = augmented_data['training_plan']
X_train_aug, y_train_aug

(     moving_time ride_count
 62   3402.138306          8
 62   3402.241408          9
 62   3402.173201          8
 62   3401.979884          9
 62   3402.213546         10
 ..           ...        ...
 102   5000.47815         13
 102  5000.575756         14
 102  5000.720474         12
 102  5000.570067         13
 102  5000.664383         12
 
 [415 rows x 2 columns],
 62     0_0
 62     0_0
 62     0_0
 62     0_0
 62     0_0
       ... 
 102    1_1
 102    1_1
 102    1_1
 102    1_1
 102    1_1
 Name: training_plan, Length: 415, dtype: object)

In [142]:
train_test_classifiers(X_train_aug, X_test, y_train_aug, y_test)


Classifier: Logistic Regression
Accuracy: 0.4722
F1 Score: 0.4722

Classifier: Decision Tree
Accuracy: 0.3056
F1 Score: 0.3056

Classifier: Random Forest
Accuracy: 0.3333
F1 Score: 0.3333

Classifier: Support Vector Machine
Accuracy: 0.3611
F1 Score: 0.3611

Classifier: K-Nearest Neighbors
Accuracy: 0.4444
F1 Score: 0.4444


In [143]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

# Define a list of classifiers
classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('Support Vector Machine', SVC()),
    ('K-Nearest Neighbors', KNeighborsClassifier())
]

# Iterate through each classifier
for name, classifier in classifiers:
    # Create a pipeline with a standard scaler and the classifier
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', classifier)
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)

    # Print the cross-validation scores
    print(f'{name} - Cross Validation Scores:', cv_scores)
    print(f'Mean Accuracy: {np.mean(cv_scores)}\n')

    # Train the model on the entire training set
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Evaluate the performance on the test set
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{name} - Accuracy on Test Set:', accuracy)
    print('\n' + '='*30 + '\n')


Logistic Regression - Cross Validation Scores: [0.47058824 0.58823529 0.47058824 0.375      0.375     ]
Mean Accuracy: 0.45588235294117646

Logistic Regression - Accuracy on Test Set: 0.4722222222222222


Decision Tree - Cross Validation Scores: [0.47058824 0.70588235 0.35294118 0.375      0.3125    ]
Mean Accuracy: 0.44338235294117645

Decision Tree - Accuracy on Test Set: 0.3055555555555556


Random Forest - Cross Validation Scores: [0.47058824 0.58823529 0.41176471 0.4375     0.4375    ]
Mean Accuracy: 0.46911764705882353

Random Forest - Accuracy on Test Set: 0.3888888888888889


Support Vector Machine - Cross Validation Scores: [0.52941176 0.64705882 0.47058824 0.4375     0.375     ]
Mean Accuracy: 0.4919117647058823

Support Vector Machine - Accuracy on Test Set: 0.4166666666666667


K-Nearest Neighbors - Cross Validation Scores: [0.52941176 0.52941176 0.47058824 0.5        0.3125    ]
Mean Accuracy: 0.46838235294117647

K-Nearest Neighbors - Accuracy on Test Set: 0.3611111111111